# iPhone case

### *Sofia Shchipinskaya*

In [18]:
# importing libraries
import os
import glob
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import resnet50, inception_v3, vgg16
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras.optimizers import Adam
import keras
from keras import backend as K
from keras.models import load_model
import tensorflow as tf
from sklearn import metrics
from keras.callbacks import LearningRateScheduler
import math
from classification_models.resnet import ResNet18
from keras.models import load_model
from keras import optimizers

## Preporation step

In [5]:
path='full dataset\\'

In [44]:
Not=glob.glob(os.path.join(path,'An/*/','*.jpg'))
Iphone=glob.glob(os.path.join(path,'Ip/*/','*.jpg'))

In [34]:
os.mkdir(os.path.join(path,'val'))
os.mkdir(os.path.join(path,'train'))
os.mkdir(os.path.join(path,'test'))
for t in ['train/','val/','test/']:
    for folder in ['An','Ip']:
        os.mkdir(os.path.join(path,t,folder))

In [45]:
files=np.append(Not,Iphone)
shuffle=np.random.permutation(len(files))

In [46]:
for i in shuffle[:2000]:
    folder=files[i].split('\\')[1]
    image=files[i].split('\\')[-1]
    os.rename(files[i],path+'val\\'+folder+'\\'+ str(i)+'.jpg')

In [47]:
for i in shuffle[2000:-2000]:
    folder=files[i].split('\\')[1]
    image=files[i].split('\\')[-1]
    os.rename(files[i],path+'train\\'+folder+'\\'+ str(i)+'.jpg')

In [48]:
for i in shuffle[-2000:]:
    folder=files[i].split('\\')[1]
    image=files[i].split('\\')[-1]
    os.rename(files[i],path+'test\\'+folder+'\\'+ str(i)+'.jpg')

## Random transformation

In [4]:
# creating random transformation
val_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        rescale=1./255,
        fill_mode='nearest')

train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        rescale=1./255,
        fill_mode='nearest')

test_datagen = ImageDataGenerator(
        rescale=1./255,
        fill_mode='nearest')

In [5]:
# creating data loaders
val_generator = val_datagen.flow_from_directory(
        'full dataset/val',  
        target_size=(224, 224),  
        batch_size=16,
        class_mode='binary')  

train_generator = val_datagen.flow_from_directory(
        'full dataset/train',  
        target_size=(224, 224),  
        batch_size=16,
        class_mode='binary')  

test_generator = val_datagen.flow_from_directory(
        'full dataset/test',  
        target_size=(224, 224),  
        batch_size=64,
        shuffle=False,
        class_mode='binary') 

Found 2000 images belonging to 2 classes.
Found 48684 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [6]:
train_generator.class_indices

{'An': 0, 'Ip': 1}

## Installing the neural network

In [7]:
# installing ResNet18
!pip install git+https://github.com/qubvel/classification_models.git

  Cloning https://github.com/qubvel/classification_models.git to /tmp/pip-req-build-6qibrunu
  Stored in directory: /tmp/pip-ephem-wheel-cache-l4u4iba6/wheels/de/2b/fd/29a6d33edb8c28bc7d94e95ea1d39c9a218ac500a3cfb1b197
Successfully built image-classifiers


In [8]:
# building the model
base_model = ResNet18(input_shape=(224,224,3), include_top=False, classes=2)

x = GlobalAveragePooling2D()(base_model.output)

predictions = Dense(512, activation='relu')(x)
predictions = Dense(2, activation='softmax')(predictions)

model = Model(inputs=[base_model.input], outputs=predictions)

for layer in base_model.layers:
    layer.trainable = True

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
# defining f1 function
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1) 

In [19]:
# defining optimizer
sgd = optimizers.SGD(lr=0.001, decay=0.0, momentum=0.0, nesterov=False)

In [20]:
# defining hyperparameters for tunning
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=sgd,
#               optimizer=Adam(lr=0.001),
              metrics=['acc',f1])

In [21]:
# defining batch sizes
batch_size=16

In [22]:
# defining sample sizes 
nb_samples_train = len(train_generator.filenames)
nb_samples_val= len(val_generator.filenames)
nb_samples_test = len(test_generator.filenames)

In [23]:
# learning rate schedule
def step_decay(epoch):
    initial_lrate=0.001
    drop=0.5
    epochs_drop=10
    lrate=initial_lrate*math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

In [24]:
# learning rate callback
lrate = LearningRateScheduler(step_decay)
callbacks_list = [lrate]

In [25]:
# training the model
model.fit_generator(
        train_generator,
        steps_per_epoch=nb_samples_train/batch_size,
        epochs=100,
        validation_data=val_generator,
        validation_steps=nb_samples_val/batch_size,
        callbacks=callbacks_list)

Epoch 1/100
3043/3042 [==============================] - 1441s 473ms/step - loss: 0.6666 - acc: 0.5840 - f1: 0.4740 - val_loss: 0.6656 - val_acc: 0.5945 - val_f1: 0.4742
Epoch 2/100
3043/3042 [==============================] - 878s 288ms/step - loss: 0.6637 - acc: 0.5904 - f1: 0.4738 - val_loss: 0.6621 - val_acc: 0.6065 - val_f1: 0.4711
Epoch 3/100
3043/3042 [==============================] - 878s 289ms/step - loss: 0.6631 - acc: 0.5915 - f1: 0.4725 - val_loss: 0.6654 - val_acc: 0.5965 - val_f1: 0.4680
Epoch 4/100
3043/3042 [==============================] - 883s 290ms/step - loss: 0.6602 - acc: 0.5959 - f1: 0.4730 - val_loss: 0.6651 - val_acc: 0.6055 - val_f1: 0.4772
Epoch 5/100
3043/3042 [==============================] - 884s 290ms/step - loss: 0.6618 - acc: 0.5944 - f1: 0.4740 - val_loss: 0.6627 - val_acc: 0.6045 - val_f1: 0.4682
Epoch 6/100
3043/3042 [==============================] - 884s 290ms/step - loss: 0.6592 - acc: 0.5992 - f1: 0.4717 - val_loss: 0.6663 - val_acc: 0.6010 - 

3043/3042 [==============================] - 886s 291ms/step - loss: 0.6461 - acc: 0.6231 - f1: 0.4755 - val_loss: 0.6439 - val_acc: 0.6280 - val_f1: 0.4769
Epoch 98/100
3043/3042 [==============================] - 886s 291ms/step - loss: 0.6471 - acc: 0.6225 - f1: 0.4764 - val_loss: 0.6497 - val_acc: 0.6225 - val_f1: 0.4732
Epoch 99/100
3043/3042 [==============================] - 883s 290ms/step - loss: 0.6458 - acc: 0.6215 - f1: 0.4752 - val_loss: 0.6464 - val_acc: 0.6265 - val_f1: 0.4710
Epoch 100/100
3043/3042 [==============================] - 882s 290ms/step - loss: 0.6474 - acc: 0.6189 - f1: 0.4755 - val_loss: 0.6479 - val_acc: 0.6235 - val_f1: 0.4723


In [26]:
# final test
y_prob = model.predict_generator(test_generator,steps = nb_samples_test/64,use_multiprocessing=True)
y_true=test_generator.labels
y_pred=y_prob.argmax(1)
final_f1=metrics.f1_score(y_true, y_pred , average='binary')
final_f1

0.6216216216216216

In [27]:
# saving the model and weights
model.save('model.h5')
model.save_weights('weights.h5')

In [28]:
# loading the model and weights
model = load_model('model.h5',custom_objects={'f1': f1})
model.load_weights('weights.h5')

In [ ]:
# calculate avarage percision
average_precision = average_precision_score(validation_generator.classes, [x[1] for x in full_pred])
print('Average precision-recall score for full model: {0:0.10f}'.format(average_precision_full))
